# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
vacation_df = pd.read_csv('../output_data/cities_test.csv')
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,High Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.10,-79.24,66.69,77,90,11.41,sh,1602371576
1,1,campo alegre,-9.78,-36.35,77.00,83,20,13.87,br,1602371718
2,2,saint anthony,45.02,-93.22,62.35,34,1,3.36,ca,1602371718
3,4,qaanaaq,77.48,-69.36,31.82,80,77,8.72,gl,1602371718
4,5,hilo,19.73,-155.09,81.36,58,75,9.17,us,1602371618


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Store latitude and longitude in locations
locations = vacation_df[['Lat','Lng']]

# Filla NaN values and convert to float
humidity = vacation_df['Humidity'].astype(float)
locations.head()

,Lat,Lng
0,42.10,-79.24
1,-9.78,-36.35
2,45.02,-93.22
3,77.48,-69.36
4,19.73,-155.09


In [17]:
# Plot Heatmap
fig = gmaps.figure()

# create heat layer
humidity_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, max_intensity = 10,
                                point_radius = 1)
# add the layer to the map
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
hotel_df = vacation_df.loc[(vacation_df['High Temp'] < 75) & (vacation_df['High Temp'] > 45)\
                             & (vacation_df['Lat'] < 75) & (vacation_df['Cloudiness'] < 40) & \
                             (vacation_df['Humidity'] < 60)]
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,High Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,saint anthony,45.02,-93.22,62.35,34,1,3.36,ca,1602371718
15,20,ondjiva,-17.07,15.73,61.48,30,0,6.98,ao,1602371722
19,24,marzuq,14.40,46.47,69.49,44,0,0.74,ly,1602371462
27,34,hobart,-42.88,147.33,55.96,58,20,12.75,au,1602371529


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
# base url to pull maps data
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# for look to find each location from the df
for hotel in hotel_df:
    lat = hotel['Lat']
    long = hotel['Lng']

    lat_long = f"{lat},{long}"   #set variable for locations lat/long. 
                                        #here's what i'm doing incorrectly
    params = {"location": lat_long,     #set parameters for search
             "radius": 5000,
             "rankby": "distance",
             "type": "lodging",
             "key": gkey,
             }
    # locate hotel
    response = requests.get(base_url, params=params).json()
    
    hotels = response['hotels']

    # hotel data into the data frame
    try:
        hotel['Hotel Name'] = results[0]['name']
    except:
        print('Not hotel in the nearby area')






TypeError: string indices must be integers

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
